In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.2.0/spark-3.2.0-bin-hadoop3.2.tgz
!tar xf spark-3.2.0-bin-hadoop3.2.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.0-bin-hadoop3.2"

In [ ]:
import findspark
findspark.init()
findspark.find()
from pyspark.sql import SparkSession


spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [ ]:
from pyspark.sql.functions import col
from pyspark.sql import functions as f
from pyspark.sql.types import IntegerType, StringType, ArrayType
from pyspark.sql.functions import monotonically_increasing_id

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ls "/content/gdrive/My Drive/Bigdata/final-exercises/data"
DATA_PATH = "/content/gdrive/My Drive/Bigdata/final-exercises/data/foodmart.csv"

Mounted at /content/gdrive
followers.txt  foodmart.csv  iris.csv  movies.json  mushrooms.csv  users.txt


In [ ]:
df = spark.read.load(DATA_PATH, format="csv", header=True, delimiter=",")
df.show(5)

+------------+---------+-------+--------------+------+---------+----+-------+-------+------------+-----------------+------+------+-----+---------+---------------+-----+--------+------+-------------+------------------+-----------+-------+-----------+--------------+--------+----------+-----------+-----------+----+------+-----------+----------+----+-----------------+---------------+------------+-------------+----------+--------------+-----------------+---+---------+----------+--------------+--------+---------+---------+---+-----+-----+----------+----+----+---------+-------+------------+----+-------+-----------+--------+------------+-----------+-----+-------------+----------------+-----+----------------+-------+---------+------------+-------------+-------------+---------+--------+----+--------+------+------------+-------+---------+------+------------+----+----+----------+------+-------+----------------+-----+----------+----+--------------+-----+------------+----+---------+-------+----+----

In [ ]:
from pyspark.sql.types import IntegerType, StringType

def func(row):
  arr = []
  for i in range(len(row)):
    if row[i] == '1':
      arr.append(cols[i])
  return ",".join(arr)

cols = df.columns
bought_list = f.udf(lambda row: func(row), StringType())

items_df = df.withColumn("Items", bought_list(f.struct([df[x] for x in df.columns])))
items_df.show(truncate=False)

+------------+---------+-------+--------------+------+---------+----+-------+-------+------------+-----------------+------+------+-----+---------+---------------+-----+--------+------+-------------+------------------+-----------+-------+-----------+--------------+--------+----------+-----------+-----------+----+------+-----------+----------+----+-----------------+---------------+------------+-------------+----------+--------------+-----------------+---+---------+----------+--------------+--------+---------+---------+---+-----+-----+----------+----+----+---------+-------+------------+----+-------+-----------+--------+------------+-----------+-----+-------------+----------------+-----+----------------+-------+---------+------------+-------------+-------------+---------+--------+----+--------+------+------------+-------+---------+------+------------+----+----+----------+------+-------+----------------+-----+----------+----+--------------+-----+------------+----+---------+-------+----+----

In [ ]:
from pyspark.sql.types import IntegerType, StringType, ArrayType

def func(row):
  arr = []
  for i in range(len(row)):
    if row[i] == '1':
      arr.append(cols[i])
  return arr

cols = df.columns
bought_list = f.udf(lambda row: func(row), ArrayType(StringType()))

items_df = df.withColumn("Items", bought_list(f.struct([df[x] for x in df.columns])))
items_df.show(truncate=False)

+------------+---------+-------+--------------+------+---------+----+-------+-------+------------+-----------------+------+------+-----+---------+---------------+-----+--------+------+-------------+------------------+-----------+-------+-----------+--------------+--------+----------+-----------+-----------+----+------+-----------+----------+----+-----------------+---------------+------------+-------------+----------+--------------+-----------------+---+---------+----------+--------------+--------+---------+---------+---+-----+-----+----------+----+----+---------+-------+------------+----+-------+-----------+--------+------------+-----------+-----+-------------+----------------+-----+----------------+-------+---------+------------+-------------+-------------+---------+--------+----+--------+------+------------+-------+---------+------+------------+----+----+----------+------+-------+----------------+-----+----------+----+--------------+-----+------------+----+---------+-------+----+----

In [ ]:
from pyspark.sql.functions import monotonically_increasing_id
processed_df = items_df.withColumn("ID", monotonically_increasing_id() + 1).select('ID', 'Items')
processed_df.show(truncate=False)

+---+---------------------------------------------------------------------+
|ID |Items                                                                |
+---+---------------------------------------------------------------------+
|1  |[Acetominifen, Cheese, Home Magazines, Shampoo]                      |
|2  |[Acetominifen, Cheese, Hard Candy, Milk, Pot Scrubbers, Rice]        |
|3  |[Coffee, Deli Salads]                                                |
|4  |[Eggs, Gum, Milk, Soup]                                              |
|5  |[Cheese, Dried Fruit, Frozen Chicken, Plastic Utensils]              |
|6  |[Shampoo]                                                            |
|7  |[Milk, Paper Wipes, Waffles]                                         |
|8  |[Donuts, Dried Fruit, Frozen Chicken]                                |
|9  |[Cooking Oil, Hamburger, Maps, Popsicles]                            |
|10 |[Cheese, Cooking Oil, Dips, Preserves, TV Dinner]                    |
|11 |[Nasal 

In [ ]:
from pyspark.ml.fpm import FPGrowth
fpGrowth = FPGrowth(itemsCol="Items", minSupport=0.1, minConfidence=0.9)
model = fpGrowth.fit(processed_df)
# get the frequent itemsets
patternsDF = model.freqItemsets
# get the association rules
rulesDF = model.associationRules

/content/spark-3.2.0-bin-hadoop3.2/python/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


In [ ]:
patternsDF.show(truncate=False)

+-------------+----+
|items        |freq|
+-------------+----+
|[Dried Fruit]|256 |
|[Soup]       |280 |
|[Cookies]    |238 |
|[Cheese]     |285 |
+-------------+----+



In [ ]:
rulesDF.show(truncate=False)

+----------+----------+----------+----+-------+
|antecedent|consequent|confidence|lift|support|
+----------+----------+----------+----+-------+
+----------+----------+----------+----+-------+

